Shreyas Vaderiyattil, Ritvik Singh, Shiv Rathod

Introduction

Formula One is an international racing league that gathers the top drivers from around the world and they compete against each other using one of the fastest cars built for racing on circuits around the world. In this tutorial we will be analyizing the lap times of various drivers as well as their pit stop times. In addition, we will take a look at these timings and compare it to their qualified place for the race and see if a driver would place above or below their qualified time.

Predicting what place a driver can place is important to not only the racing team and driver but also fans that follow the sport. Racing teams and drivers find this useful for their approach and strategy for the race. This can help with fast decisions that are needed during a race between the driver and the team